In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np

from tqdm import tqdm

torch.random.manual_seed(42)
np.random.seed(42)

In [ ]:
class_to_superclass = {
    49: 10, 33: 10, 72: 0, 51: 4, 71: 10, 92: 2, 15: 11, 14: 7, 23: 10,
    0: 4, 75: 12, 81: 19, 69: 19, 40: 5, 43: 8, 97: 8, 70: 2, 53: 4, 29: 15,
    21: 11, 16: 3, 39: 5, 8: 18, 20: 6, 61: 3, 41: 19, 93: 15, 56: 17, 73: 1,
    58: 18, 11: 14, 25: 6, 37: 9, 63: 12, 24: 7, 22: 5, 17: 9, 4: 0, 6: 7, 9: 3,
    57: 4, 2: 14, 32: 1, 52: 17, 42: 8, 77: 13, 27: 15, 65: 16, 7: 7, 35: 14, 82: 2,
    66: 12, 90: 18, 67: 1, 91: 1, 10: 3, 78: 15, 54: 2, 89: 19, 18: 7, 13: 18, 50: 16,
    26: 13, 83: 4, 47: 17, 95: 0, 76: 9, 59: 17, 85: 19, 19: 11, 46: 14, 1: 1, 74: 16,
    60: 10, 64: 12, 45: 13, 36: 16, 87: 5, 30: 0, 99: 13, 80: 16, 28: 3, 98: 14, 12: 9,
    94: 6, 68: 9, 44: 15, 31: 11, 79: 13, 34: 12, 55: 0, 62: 2, 96: 17, 84: 6, 38: 11,
    86: 5, 5: 6, 48: 18, 3: 8, 88: 8
}

# Top-1 Accuracy
def top1_accuracy(outputs, targets):
    """Compute the Top-1 accuracy"""
    _, predicted = outputs.max(1)
    correct = predicted.eq(targets).sum().item()
    accuracy = correct / targets.size(0)
    return accuracy * 100

# Top-5 Accuracy
def top5_accuracy(outputs, targets):
    """Compute the Top-5 accuracy"""
    _, predicted = outputs.topk(5, dim=1)
    correct = predicted.eq(targets.view(-1, 1).expand_as(predicted)).sum().item()
    accuracy = correct / targets.size(0)
    return accuracy * 100

# Superclass Accuracy
def superclass_accuracy(outputs, targets, class_to_superclass=class_to_superclass):
    """Compute the Superclass accuracy"""
    _, predicted = outputs.max(1)
    predicted_superclasses = torch.tensor([class_to_superclass[p.item()] for p in predicted])
    target_superclasses = torch.tensor([class_to_superclass[t.item()] for t in targets])
    correct = predicted_superclasses.eq(target_superclasses).sum().item()
    accuracy = correct / targets.size(0)
    return accuracy * 100

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from tqdm import tqdm

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1   = nn.BatchNorm2d(planes)
        self.relu  = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2   = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride !=1 or in_planes != planes * self.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes * self.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * self.expansion)
            )

    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(identity)
        out = self.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=100):  # 设置为100类
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1  = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1    = nn.BatchNorm2d(64)
        self.relu   = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(block, 64,  num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.avgpool= nn.AdaptiveAvgPool2d((1,1))
        self.fc     = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks - 1)
        layers  = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out

def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])

# Hyperparameters and settings
batch_size = 64
learning_rate = 1e-3
weight_decay = 1e-4
epochs = 200
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data Augmentation for Training
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408),
                         (0.2675, 0.2565, 0.2761)),
])

# Data Transformation for Validation (NO Augmentation)
val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762)),
])

cifar100_full = datasets.CIFAR100(root='./data', train=True, download=True, transform=None)

# Stratified split for train and validation
targets = np.array(cifar100_full.targets)
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42)
train_indices, val_indices = next(sss.split(np.zeros(len(targets)), targets))

# Create train and validation subsets with appropriate transformations
train_dataset = torch.utils.data.Subset(datasets.CIFAR100(root='./data', train=True, download=True, transform=train_transform), train_indices)
val_dataset = torch.utils.data.Subset(datasets.CIFAR100(root='./data', train=True, download=True, transform=val_transform), val_indices)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Model, loss function, and optimizer
model = ResNet18().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

# Define where to save your model
best_model_path = 'best_model_resnet.pth'
best_val_loss = float('inf')

train_losses = []
train_top1 = []
train_top5 = []
train_superclass = []

val_losses = []
val_top1 = []
val_top5 = []
val_superclass = []

# Training and validation
def train(epoch):
    model.train()
    train_loss, correct, total = 0, 0, 0
    top1_acc, top5_acc, superclass_acc = 0, 0, 0
    for inputs, targets in tqdm(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)

        # Calculate accuracies
        top1_acc += top1_accuracy(outputs, targets)
        top5_acc += top5_accuracy(outputs, targets)
        superclass_acc += superclass_accuracy(outputs, targets, class_to_superclass)

    # Calculate average accuracy over all batches
    loss_avg = train_loss / len(train_loader)
    top1_avg = top1_acc / len(train_loader)
    top5_avg = top5_acc / len(train_loader)
    superclass_avg = superclass_acc / len(train_loader)

    train_losses.append(loss_avg)
    train_top1.append(top1_avg)
    train_top5.append(top5_avg)
    train_superclass.append(superclass_avg)

    print('')
    print(f'Epoch {epoch} | Train Loss: {loss_avg:.4f} | Top-1 Acc: {top1_avg:.2f}% | Top-5 Acc: {top5_avg:.2f}% | Superclass Acc: {superclass_avg:.2f}%', end=' ')

def validate(epoch):
    global best_val_loss
    model.eval()
    val_loss, correct, total = 0, 0, 0
    top1_acc, top5_acc, superclass_acc = 0, 0, 0

    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)

            # Calculate accuracies
            top1_acc += top1_accuracy(outputs, targets)
            top5_acc += top5_accuracy(outputs, targets)
            superclass_acc += superclass_accuracy(outputs, targets, class_to_superclass)

    # Calculate average accuracy over all batches
    loss_avg = val_loss / len(val_loader)
    top1_avg = top1_acc / len(val_loader)
    top5_avg = top5_acc / len(val_loader)
    superclass_avg = superclass_acc / len(val_loader)

    val_losses.append(loss_avg)
    val_top1.append(top1_avg)
    val_top5.append(top5_avg)
    val_superclass.append(superclass_avg)

    print(f'| Val Loss: {loss_avg:.4f} | Top-1 Acc: {top1_avg:.2f}% | Top-5 Acc: {top5_avg:.2f}% | Superclass Acc: {superclass_avg:.2f}%')
    # Checkpoint: Save model if validation loss is the best seen so far
    if loss_avg < best_val_loss:
        print(f'Validation loss improved by {(best_val_loss - loss_avg):.4f}. Saving model...')
        best_val_loss = loss_avg
        torch.save(model.state_dict(), best_model_path)

for epoch in range(1, epochs + 1):
    train(epoch)
    validate(epoch)
    scheduler.step()

torch.save(model.state_dict(), 'resnet_variant_cifar100.pth')

100%|██████████| 169001437/169001437 [00:18<00:00, 9101471.11it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 625/625 [00:21<00:00, 29.43it/s]


Epoch 1 | Train Loss: 4.1035 | Top-1 Acc: 7.51% | Top-5 Acc: 24.66% | Superclass Acc: 18.10% 

| Val Loss: 3.5874 | Top-1 Acc: 13.94% | Top-5 Acc: 39.17% | Superclass Acc: 28.33%
Validation loss improved by inf. Saving model...


...


Epoch 199 | Train Loss: 0.0614 | Top-1 Acc: 98.57% | Top-5 Acc: 99.75% | Superclass Acc: 98.92% 

| Val Loss: 1.2734 | Top-1 Acc: 70.43% | Top-5 Acc: 89.93% | Superclass Acc: 80.46%


100%|██████████| 625/625 [00:19<00:00, 31.75it/s]


Epoch 200 | Train Loss: 0.0595 | Top-1 Acc: 98.62% | Top-5 Acc: 99.76% | Superclass Acc: 98.97% 

| Val Loss: 1.2743 | Top-1 Acc: 70.40% | Top-5 Acc: 89.84% | Superclass Acc: 80.44%


In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762)),
])

test_cifar100 = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_cifar100, batch_size=batch_size, shuffle=False, num_workers=4)

Files already downloaded and verified


In [ ]:
class_to_superclass = {
    49: 10, 33: 10, 72: 0, 51: 4, 71: 10, 92: 2, 15: 11, 14: 7, 23: 10,
    0: 4, 75: 12, 81: 19, 69: 19, 40: 5, 43: 8, 97: 8, 70: 2, 53: 4, 29: 15,
    21: 11, 16: 3, 39: 5, 8: 18, 20: 6, 61: 3, 41: 19, 93: 15, 56: 17, 73: 1,
    58: 18, 11: 14, 25: 6, 37: 9, 63: 12, 24: 7, 22: 5, 17: 9, 4: 0, 6: 7, 9: 3,
    57: 4, 2: 14, 32: 1, 52: 17, 42: 8, 77: 13, 27: 15, 65: 16, 7: 7, 35: 14, 82: 2,
    66: 12, 90: 18, 67: 1, 91: 1, 10: 3, 78: 15, 54: 2, 89: 19, 18: 7, 13: 18, 50: 16,
    26: 13, 83: 4, 47: 17, 95: 0, 76: 9, 59: 17, 85: 19, 19: 11, 46: 14, 1: 1, 74: 16,
    60: 10, 64: 12, 45: 13, 36: 16, 87: 5, 30: 0, 99: 13, 80: 16, 28: 3, 98: 14, 12: 9,
    94: 6, 68: 9, 44: 15, 31: 11, 79: 13, 34: 12, 55: 0, 62: 2, 96: 17, 84: 6, 38: 11,
    86: 5, 5: 6, 48: 18, 3: 8, 88: 8
}

# Top-1 Accuracy
def top1_accuracy(outputs, targets):
    """Compute the Top-1 accuracy"""
    _, predicted = outputs.max(1)
    correct = predicted.eq(targets).sum().item()
    accuracy = correct / targets.size(0)
    return accuracy * 100

# Top-5 Accuracy
def top5_accuracy(outputs, targets):
    """Compute the Top-5 accuracy"""
    _, predicted = outputs.topk(5, dim=1)
    correct = predicted.eq(targets.view(-1, 1).expand_as(predicted)).sum().item()
    accuracy = correct / targets.size(0)
    return accuracy * 100

# Superclass Accuracy
def superclass_accuracy(outputs, targets, class_to_superclass=class_to_superclass):
    """Compute the Superclass accuracy"""
    _, predicted = outputs.max(1)
    predicted_superclasses = torch.tensor([class_to_superclass[p.item()] for p in predicted])
    target_superclasses = torch.tensor([class_to_superclass[t.item()] for t in targets])
    correct = predicted_superclasses.eq(target_superclasses).sum().item()
    accuracy = correct / targets.size(0)
    return accuracy * 100

def test_evaluate(model, dataloader, device):
    model.eval()
    val_loss, correct, total = 0, 0, 0
    top1_acc, top5_acc, superclass_acc = 0, 0, 0

    with torch.no_grad():
        for inputs, targets in tqdm(dataloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)

            loss = criterion(outputs, targets)

            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)

            # Calculate accuracies
            top1_acc += top1_accuracy(outputs, targets)
            top5_acc += top5_accuracy(outputs, targets)
            superclass_acc += superclass_accuracy(outputs, targets, class_to_superclass)

    # Calculate average accuracy over all batches
    top1_avg = top1_acc / len(dataloader)
    top5_avg = top5_acc / len(dataloader)
    superclass_avg = superclass_acc / len(dataloader)

    print(f'| Test Loss: {val_loss/len(dataloader):.4f} | Test Top-1 Acc: {top1_avg:.2f}% | Test Top-5 Acc: {top5_avg:.2f}% | Test Superclass Acc: {superclass_avg:.2f}%')
    return top1_avg, top5_avg, superclass_avg

In [ ]:
top1_test, top5_test, superclass_test = test_evaluate(model, test_loader, device)
print(f'Accuracy Test: {top1_test}, Top 5 Test: {top5_test}, Superclass Accuracy Test: {superclass_test}')

100%|██████████| 157/157 [00:01<00:00, 105.50it/s]

| Test Loss: 1.2726 | Test Top-1 Acc: 70.71% | Test Top-5 Acc: 89.50% | Test Superclass Acc: 80.18%
Accuracy Test: 70.71058917197452, Top 5 Test: 89.50039808917198, Superclass Accuracy Test: 80.17515923566879
